In [ ]:
# Option 1: Install the full training bundle (Recommended)
!pip install "mlx-lm[train]"

# Option 2: Just install the missing library
!pip install datasets

In [1]:
!python -c "import datasets; print('Datasets library found!')"

Datasets library found!


In [1]:
!python -m mlx_lm.lora \
  --model mlx-community/gemma-3-12b-it-4bit \
  --train \
  --data ./anime_data \
  --iters 500 \
  --batch-size 1 \
  --learning-rate 1e-5 \
  --adapter-path ./anime_adapters

Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
Loading pretrained model
Fetching 13 files: 100%|█████████████████████| 13/13 [00:00<00:00, 14637.84it/s]
Loading datasets
Training
Trainable parameters: 0.014% (1.769M/12772.913M)
Starting training..., iters: 500
Calculating loss...: 100%|████████████████████████| 2/2 [00:07<00:00,  3.71s/it]
Iter 1: Val loss 9.129, Val took 7.461s
Iter 10: Train loss 6.189, Learning Rate 1.000e-05, It/sec 0.279, Tokens/sec 21.517, Trained Tokens 772, Peak mem 8.199 GB
Iter 20: Train loss 4.356, Learning Rate 1.000e-05, It/sec 0.294, Tokens/sec 22.661, Trained Tokens 1544, Peak mem 8.199 GB
Iter 30: Train loss 3.503, Learning Rate 1.000e-05, It/sec 0.291, Tokens/sec 22.488, Trained Tokens 2316, Peak mem 8.199 GB
Iter 40: Train loss 2.887, Learning Rate 1.000e-05, It/sec 0.272, Tokens/sec 20.961, Trained Tokens 3088, Peak mem 8.199 GB
Iter 50: Train loss 2.540, Learning Rate 1.000e-05

In [ ]:
from mlx_lm import load, generate

# 1. Load the model (the 4-bit version is essential for M1 RAM efficiency)
model_name = "mlx-community/gemma-3-12b-it-4bit"
model, tokenizer = load(model_name, adapter_path="./anime_adapters")

# 2. Initialize an empty conversation history
messages = []

def chat_with_memory():
    print("--- Gemma (Fine-tuned) Chatbot (Type 'clear' to reset or 'exit' to stop) ---")
    
    while True:
        # Who are the main characters of GXYZ123?
        # What's Rudo's vital instrument in anime Gachiakuta?
        user_input = input("\nYou: ")
        
        if user_input.lower() == 'exit':
            break
        if user_input.lower() == 'clear':
            messages.clear()
            print("History cleared!")
            continue

        # Add user message to history
        messages.append({"role": "user", "content": user_input})

        # Format history into the Gemma-specific string format
        # This adds the <start_of_turn> and <end_of_turn> tags automatically
        prompt = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )

        # Generate and stream the response
        print("\nGemma (Fine-tuned): ", end="", flush=True)
        response = generate(
            model, 
            tokenizer, 
            prompt=prompt, 
            max_tokens=1000, 
            verbose=True # Streaming enabled
        )
        
        # Save the model's response to history for next time
        messages.append({"role": "assistant", "content": response})

# Run the chat
chat_with_memory()

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

--- Gemma (Fine-tuned) Chatbot (Type 'clear' to reset or 'exit' to stop) ---



You:  Who are the main characters of GXYZ123?



==========e-tuned): 
The primary characters include 'Protocol-X,' a sentient glitch trying to restore its memory, and 'User-774,' an anonymous moderator who discovers a hidden world within the site's code.
Prompt: 21 tokens, 10.108 tokens-per-sec
Generation: 42 tokens, 8.263 tokens-per-sec
Peak memory: 7.328 GB
